# 8. Filtering 2 - Function Word selection

#### This script loads the Function Words dataset and selects the most relevant ones, so as to not have too many sprseness in the data

### Imports

In [1]:
import pandas as pd
import numpy as np

### Definitions

In [2]:
all_fw_filename = 'data/FunctionWords-extracted.csv'

selected_fw_filename = 'data/FunctionWords-selected.txt'

### Load Bleached bigrams

In [3]:
df = pd.read_csv(all_fw_filename)

df.head()

,Word,FrequencyMale,FrequencyFemale
0,i,1567.799083,2414.589971
1,we,758.887689,872.903331
2,you,1962.369570,2229.430293
3,he,535.290889,676.134495
4,she,307.796310,449.041207


### Get information on what is considered as 'low frequency'

In [4]:
total_count = np.asarray(df['FrequencyMale'] + df['FrequencyFemale'])

for i in [1, 10, 20, 50, 100, 1000, 10000]:
    print('Counts more than {0}: {1}'.format(i, np.sum(total_count > i)))

Counts more than 1: 251
Counts more than 10: 227
Counts more than 20: 212
Counts more than 50: 174
Counts more than 100: 148
Counts more than 1000: 46
Counts more than 10000: 2


We will consider only those more than 50

### Remove columns with low frequency

In [5]:
df = df[total_count > 50]

total_count = total_count[total_count > 50]

### Calculate the entropy column

In [6]:
male_prob, female_prob = df['FrequencyMale']/total_count, df['FrequencyFemale']/total_count

male_prob.shape
df['Entropy'] = - ( male_prob * np.log2(male_prob) + female_prob * np.log2(female_prob) )

df.head()

,Word,FrequencyMale,FrequencyFemale,Entropy
0,i,1567.799083,2414.589971,0.967135
1,we,758.887689,872.903331,0.996475
2,you,1962.369570,2229.430293,0.997070
3,he,535.290889,676.134495,0.990227
4,she,307.796310,449.041207,0.974728


### Take top 50 with least entropy (ie which discriminate the most)

In [7]:
df = df.sort_values('Entropy').reset_index(drop = True)

df = df.iloc[:50]

df.head()

,Word,FrequencyMale,FrequencyFemale,Entropy
0,him,214.292223,356.036603,0.954974
1,against,51.037687,30.730540,0.955040
2,i,1567.799083,2414.589971,0.967135
3,she,307.796310,449.041207,0.974728
4,alone,32.569053,46.736906,0.976854


### Save to new text file

In [8]:
with open(selected_fw_filename, 'w') as f:
    for word in df['Word']:
        f.write(word + '\n')